<a href="https://colab.research.google.com/github/2303A52016/GEN-AI/blob/main/GenAI_TransferLearningProject_Team7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing tensorflow package**

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


























































































#**Mounting Drive and Creating directories for the Images**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/MyDrive/DogEmotions/Images/training_set'  # Path to the training dataset

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,  # Apply random shear transformation
    zoom_range=0.2,  # Apply random zoom
    horizontal_flip=True  # Randomly flip images horizontally
)

training_set = train_datagen.flow_from_directory(
    data_dir,  # Directory containing class-wise image folders
    target_size=(64, 64),  # Resize all images to 64x64 pixels
    batch_size=32,  # Number of images per batch
    class_mode='categorical'  # Use binary labels (0 or 1) for two classes
)

Found 4000 images belonging to 4 classes.


In [5]:
test_dir = '/content/drive/MyDrive/DogEmotions/Images/test_set'  # Path to the test dataset

test_datagen = ImageDataGenerator(rescale=1./255)  # Only normalize, no augmentation for test data

test_set = test_datagen.flow_from_directory(
    test_dir,  # Directory containing test images in class-wise folders
    target_size=(64, 64),  # Resize all images to 64x64 pixels
    batch_size=32,  # Number of images per batch
    class_mode='categorical'  # Binary classification (0 or 1)
)

Found 800 images belonging to 4 classes.


#**Building the MobileNetV2 Model**

In [6]:
from tensorflow.keras.applications import MobileNetV2  # Import MobileNetV2

In [7]:
base_model = MobileNetV2(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
base_model.trainable = False

<ipython-input-7-315a82fbcfc2>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(64, 64, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Using GlobalAveragePooling2D() to Flattening the 3D Model**

In [8]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

###**Adding 1st Layer**

In [9]:
x = tf.keras.layers.Dense(256, activation='relu')(x)

###**Adding Layer**

In [10]:
x = tf.keras.layers.Dense(128, activation='relu')(x)

###**Adding Layer**

In [11]:
x = tf.keras.layers.Dense(256, activation='relu')(x)

###**Adding a output layer**

In [12]:
output = tf.keras.layers.Dense(4, activation='softmax')(x)

###**Compiling and fitting the value**

In [13]:
mobilenetv2 = tf.keras.Model(inputs=base_model.input, outputs=output)

In [14]:
mobilenetv2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
mobilenetv2.fit(
    training_set,
    epochs=10,
    validation_data=test_set
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1278s 10s/step - accuracy: 0.3275 - loss: 1.4208 - val_accuracy: 0.3537 - val_loss: 1.3458
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 341ms/step - accuracy: 0.4012 - loss: 1.2964 - val_accuracy: 0.3713 - val_loss: 1.3486
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 333ms/step - accuracy: 0.4301 - loss: 1.2370 - val_accuracy: 0.3800 - val_loss: 1.3541
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 83s 339ms/step - accuracy: 0.4578 - loss: 1.1938 - val_accuracy: 0.4000 - val_loss: 1.3326
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 340ms/step - accuracy: 0.4908 - loss: 1.1478 - val_accuracy: 0.3850 - val_loss: 1.3256
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 82s 340ms/step - accuracy: 0.5287 - loss: 1.0990 - val_accuracy: 0.3725 - val_loss: 1.3623
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 345ms/step - accuracy: 0.5505 - loss: 1.0441 - val_accuracy: 0.3725 - val_loss: 1.4213
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 336ms/step - accuracy: 0.5550 - loss: 1

#**Predicting the answer**

In [16]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img(
    '/content/drive/MyDrive/Deep_Learning/DL/CNN/dataset/single_prediction/cat_or_dog_1.jpg',
    target_size=(64, 64)
)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

In [17]:
result = mobilenetv2.predict(test_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [18]:
class_names = ['angry', 'happy', 'relaxed', 'sad']

In [19]:
predicted_class = class_names[np.argmax(result)]

In [20]:
print("Predicted Emotion:", predicted_class)

Predicted Emotion: relaxed
